In [204]:
import pandas as pd
import numpy as np
import re
import string

import xgboost as xgb

from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score


import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')


In [205]:
l_dataset_aug = pd.read_csv("../source-data/L_augment.csv")
c_dataset = pd.read_csv("../source-data/C.csv", delimiter=";", encoding="windows-1251")

In [206]:
l_dataset_aug['Desc and Group'] = l_dataset_aug['Desc'] + '. ' + l_dataset_aug['Group']

In [207]:
def convert_to_lower(text):
    return text.lower()
    
def remove_numbers(text):
    return re.sub(pattern=r'\d+', repl=" ", string=text)
    
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def remove_stopwords(text):
    removed = []
    stop_words = list(stopwords.words("russian"))
    tokens = word_tokenize(text)
    for i in range(len(tokens)):
        if tokens[i] not in stop_words:
            removed.append(tokens[i])
    return " ".join(removed)

def remmove_extra_spaces(text):
    single_char_pattern = r'\s+[a-zA-Z]\s+'
    without_sc = re.sub(pattern=single_char_pattern, repl=" ", string=text)
    return without_sc

def lemmatizing(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    for i in range(len(tokens)):
        lemma_word = lemmatizer.lemmatize(tokens[i])
        tokens[i] = lemma_word
    return " ".join(tokens)

def preprocessing_desc():
    l_dataset_aug['Desc clean'] = l_dataset_aug['Desc'].apply(lambda x: convert_to_lower(x))
    l_dataset_aug['Desc clean'] = l_dataset_aug['Desc clean'].apply(lambda x: remove_numbers(x))
    l_dataset_aug['Desc clean'] = l_dataset_aug['Desc clean'].apply(lambda x: remove_punctuation(x))
    l_dataset_aug['Desc clean'] = l_dataset_aug['Desc clean'].apply(lambda x: remove_stopwords(x))
    l_dataset_aug['Desc clean'] = l_dataset_aug['Desc clean'].apply(lambda x: remmove_extra_spaces(x))
    l_dataset_aug['Desc clean'] = l_dataset_aug['Desc clean'].apply(lambda x: lemmatizing(x))
    l_dataset_aug['Length after clean'] = l_dataset_aug['Desc clean'].apply(lambda x: len(x))

def preprocessing_desc_and_group():
    l_dataset_aug['Desc Group clean'] = l_dataset_aug['Desc and Group'].apply(lambda x: convert_to_lower(x))
    l_dataset_aug['Desc Group clean'] = l_dataset_aug['Desc Group clean'].apply(lambda x: remove_numbers(x))
    l_dataset_aug['Desc Group clean'] = l_dataset_aug['Desc Group clean'].apply(lambda x: remove_punctuation(x))
    l_dataset_aug['Desc Group clean'] = l_dataset_aug['Desc Group clean'].apply(lambda x: remove_stopwords(x))
    l_dataset_aug['Desc Group clean'] = l_dataset_aug['Desc Group clean'].apply(lambda x: remmove_extra_spaces(x))
    l_dataset_aug['Desc Group clean'] = l_dataset_aug['Desc Group clean'].apply(lambda x: lemmatizing(x))

def preprocessing_desc_c():
    c_dataset['Desc clean'] = c_dataset['Desc'].apply(lambda x: convert_to_lower(x))
    c_dataset['Desc clean'] = c_dataset['Desc clean'].apply(lambda x: remove_numbers(x))
    c_dataset['Desc clean'] = c_dataset['Desc clean'].apply(lambda x: remove_punctuation(x))
    c_dataset['Desc clean'] = c_dataset['Desc clean'].apply(lambda x: remove_stopwords(x))
    c_dataset['Desc clean'] = c_dataset['Desc clean'].apply(lambda x: remmove_extra_spaces(x))
    c_dataset['Desc clean'] = c_dataset['Desc clean'].apply(lambda x: lemmatizing(x))
    c_dataset['Length after clean'] = c_dataset['Desc clean'].apply(lambda x: len(x))

def preprocessing_desc_and_group_c():
    c_dataset['Desc Group clean'] = c_dataset['Desc and Group'].apply(lambda x: convert_to_lower(x))
    c_dataset['Desc Group clean'] = c_dataset['Desc Group clean'].apply(lambda x: remove_numbers(x))
    c_dataset['Desc Group clean'] = c_dataset['Desc Group clean'].apply(lambda x: remove_punctuation(x))
    c_dataset['Desc Group clean'] = c_dataset['Desc Group clean'].apply(lambda x: remove_stopwords(x))
    c_dataset['Desc Group clean'] = c_dataset['Desc Group clean'].apply(lambda x: remmove_extra_spaces(x))
    c_dataset['Desc Group clean'] = c_dataset['Desc Group clean'].apply(lambda x: lemmatizing(x))

def preprocessing_desc_and_group_c2():
    c_dataset_v2['Desc Group clean'] = c_dataset_v2['Desc and Group'].apply(lambda x: convert_to_lower(x))
    c_dataset_v2['Desc Group clean'] = c_dataset_v2['Desc Group clean'].apply(lambda x: remove_numbers(x))
    c_dataset_v2['Desc Group clean'] = c_dataset_v2['Desc Group clean'].apply(lambda x: remove_punctuation(x))
    c_dataset_v2['Desc Group clean'] = c_dataset_v2['Desc Group clean'].apply(lambda x: remove_stopwords(x))
    c_dataset_v2['Desc Group clean'] = c_dataset_v2['Desc Group clean'].apply(lambda x: remmove_extra_spaces(x))
    c_dataset_v2['Desc Group clean'] = c_dataset_v2['Desc Group clean'].apply(lambda x: lemmatizing(x))

In [208]:
preprocessing_desc()
preprocessing_desc_and_group()

In [209]:
x_train = l_dataset_aug["Desc clean"]
x_train_cat_degr = l_dataset_aug["Desc Group clean"]

## Preprocessing for C.csv

In [210]:
preprocessing_desc_c()

In [211]:
x_test_group = c_dataset["Desc clean"]
x_test_group.shape

(28978,)

In [212]:
y_train_group = l_dataset_aug["Group"] 
y_train_group.shape

(62666,)

In [213]:
y_train_cat = l_dataset_aug["Cat"] 
y_train_cat.shape

(62666,)

In [214]:
# Label encoding for target group
lb_encoding_group = preprocessing.LabelEncoder()
y_train_group_encoded = lb_encoding_group.fit_transform(y_train_group)
y_train_group_encoded[:5]

array([19,  8,  9,  4,  9])

In [215]:
# Label encoding for target category
lb_encoding_cat = preprocessing.LabelEncoder()
y_train_cat_encoded = lb_encoding_cat.fit_transform(y_train_cat)
y_train_cat_encoded[:5]

array([72, 46, 46, 46, 46])

# Load trained model

In [216]:
import joblib

In [217]:
ctv = CountVectorizer(analyzer='word',
                         token_pattern=r'\w{1,}',
                         ngram_range=(1, 3),
                         # stop_words = 'russian'
                     )
# Fitting CountVectorizer
ctv.fit(list(x_train))

x_test_group_ctv = ctv.transform(x_test_group.values)

## Logistic Regression -> A.csv

In [218]:
clf_ctv_group = joblib.load('../model/LogisticRegression/ctv/clf_ctv_group.sav')

**Predict for group test**

In [219]:
predicted_ctv_group_test = clf_ctv_group.predict_proba(x_test_group_ctv)
classify_group_test = [np.argmax(x) for x in predicted_ctv_group_test]

In [220]:
c_dataset["Group"] = lb_encoding_group.inverse_transform(classify_group_test)

**Predict for cat test**

In [221]:
c_dataset['Desc and Group'] = c_dataset['Desc'] + '. ' + c_dataset['Group']

In [222]:
preprocessing_desc_and_group_c()
# After preprocessing
c_dataset['Desc Group clean'][:5]

0                            застряли лифте лифты
1          течь батареи спальной управление домом
2             течь сверху стояку управление домом
3               течь кнс подвале управление домом
4    течь кровли забита ливневка управление домом
Name: Desc Group clean, dtype: object

In [223]:
## Get train data from desc and group for predict category
x_test_cat = c_dataset["Desc Group clean"]
x_test_cat.shape

(28978,)

In [224]:
x_test_cat_ctv = ctv.transform(x_test_cat.values)

In [225]:
clf_ctv_cat_new = joblib.load('../model/LogisticRegression/ctv/clf_ctv_cat_new.sav')

In [226]:
predicted_ctv_cat_new_test = clf_ctv_cat_new.predict_proba(x_test_cat_ctv)
classify_cat_new_test = [np.argmax(x) for x in predicted_ctv_cat_new_test]

In [227]:
c_dataset["Cat"] = lb_encoding_cat.inverse_transform(classify_cat_new_test)

In [228]:
c_dataset = c_dataset.drop(columns=["Length after clean", "Desc clean", "Desc and Group", "Desc Group clean"], axis=1)

In [229]:
c_dataset.to_csv('../result/A.csv', index=False, encoding="windows-1251")

In [230]:
# c_dataset = pd.read_csv("../source-data/C.csv", delimiter=";", encoding="windows-1251")

## XGBoost -> A2.csv

In [231]:
c_dataset_v2 = pd.read_csv("../source-data/C.csv", delimiter=";", encoding="windows-1251")

In [232]:
clf_xg_ctv_group = joblib.load('../model/XGBoost/ctv/clf_xg_ctv_group.sav')

**Predict for group test**

In [233]:
predicted_ctv_group_test = clf_xg_ctv_group.predict_proba(x_test_group_ctv)
classify_group_test = [np.argmax(x) for x in predicted_ctv_group_test]

In [234]:
c_dataset_v2["Group"] = lb_encoding_group.inverse_transform(classify_group_test)

**Predict for category test**

In [235]:
clf_xg_ctv_cat_new = joblib.load('../model/XGBoost/ctv/clf_xg_ctv_cat_new.sav')

In [236]:
c_dataset_v2['Desc and Group'] = c_dataset_v2['Desc'] + '. ' + c_dataset_v2['Group']

In [237]:
preprocessing_desc_and_group_c2()
# After preprocessing
c_dataset_v2['Desc Group clean'][:5]

0                            застряли лифте лифты
1          течь батареи спальной управление домом
2             течь сверху стояку управление домом
3               течь кнс подвале управление домом
4    течь кровли забита ливневка управление домом
Name: Desc Group clean, dtype: object

In [238]:
## Get train data from desc and group for predict category
x_test_cat = c_dataset_v2["Desc Group clean"]
x_test_cat.shape

(28978,)

In [239]:
x_test_cat_ctv = ctv.transform(x_test_cat.values)

In [240]:
predicted_ctv_cat_new_test = clf_xg_ctv_cat_new.predict_proba(x_test_cat_ctv)
classify_cat_new_test = [np.argmax(x) for x in predicted_ctv_cat_new_test]

In [241]:
c_dataset_v2["Cat"] = lb_encoding_cat.inverse_transform(classify_cat_new_test)

In [242]:
c_dataset_v2.head()

,Number,Date,UID,Desc,Group,Cat,Desc and Group,Desc Group clean
0,1346291,01.01.2023 2:10,e87b2cfa-6159-4aa2-b789-3974a01e0b0a,Застряли в лифте,Лифты,Человек застрял в лифте,Застряли в лифте. Лифты,застряли лифте лифты
1,1346298,01.01.2023 9:09,d6257b7b-d68c-449f-aef3-6c4af1a2d78f,Течь батареи в спальной,Управление домом,Сантехника,Течь батареи в спальной . Управление домом,течь батареи спальной управление домом
2,1346301,01.01.2023 10:37,927aeffe-0842-4894-8ada-0a91ccdf1692,Течь сверху по стояку,Управление домом,Сантехника,Течь сверху по стояку . Управление домом,течь сверху стояку управление домом
3,1346303,01.01.2023 10:56,9884e4a6-2ce4-485b-913d-8e75c7779302,Течь КНС в 3 подвале,Управление домом,Канализация,Течь КНС в 3 подвале . Управление домом,течь кнс подвале управление домом
4,1346306,01.01.2023 11:44,4593c91a-8b75-46e4-83e5-557d730abea0,Течь кровли (забита ливневка),Управление домом,Кровля (Протечка),Течь кровли (забита ливневка). Управление домом,течь кровли забита ливневка управление домом


In [243]:
c_dataset_v2 = c_dataset_v2.drop(columns=["Desc and Group", "Desc Group clean"], axis=1)

In [244]:
c_dataset_v2

,Number,Date,UID,Desc,Group,Cat
0,1346291,01.01.2023 2:10,e87b2cfa-6159-4aa2-b789-3974a01e0b0a,Застряли в лифте,Лифты,Человек застрял в лифте
1,1346298,01.01.2023 9:09,d6257b7b-d68c-449f-aef3-6c4af1a2d78f,Течь батареи в спальной,Управление домом,Сантехника
2,1346301,01.01.2023 10:37,927aeffe-0842-4894-8ada-0a91ccdf1692,Течь сверху по стояку,Управление домом,Сантехника
3,1346303,01.01.2023 10:56,9884e4a6-2ce4-485b-913d-8e75c7779302,Течь КНС в 3 подвале,Управление домом,Канализация
4,1346306,01.01.2023 11:44,4593c91a-8b75-46e4-83e5-557d730abea0,Течь кровли (забита ливневка),Управление домом,Кровля (Протечка)
...,...,...,...,...,...,...
28973,1425637,10.09.2023 17:31,e637470f-6fd2-41fc-94af-7c7a83a66b60,"Течь из стыка КНС в подвале, проверить засор в...",Управление домом,Канализация
28974,1425638,10.09.2023 17:50,04e668c0-e11d-44e8-8ee3-f6154b129f86,Нет напора ХВС и совсем пропадает.,Управление домом,Сантехника
28975,1425640,10.09.2023 17:54,62f5beff-2753-490c-8115-01cfa320ee40,Обратная тяга вентканала в ванной,Управление домом,Ремонт дымовых и вентиляционных каналов
28976,1425641,10.09.2023 17:55,5f150ce4-dbcc-4cbf-9adb-dde8d4c05aba,Х/ГВС,Управление домом,ГВС (Несоответствие температурных параметров)


In [245]:
c_dataset_v2.to_csv('../result/A2.csv', index=False, encoding="windows-1251")

In [246]:
c_dataset.Group.value_counts(), c_dataset.Cat.value_counts()

(Group
 Управление домом    25880
 Домофоны             1083
 ГВС                  1081
 Лифты                 929
 Водоотведение           2
 Благоустройство         2
 Сантехника              1
 Name: count, dtype: int64,
 Cat
 Сантехника                            9493
 Общие вопросы                         3740
 Канализация                           2891
 Электроэнергия                        2493
 Опломбировка ИПУ                      2205
                                       ... 
 Забит мусоропровод                       3
 Проверка канализации                     2
 Замена арматуры с/б                      2
 Протечки в отдельных местах кровли       1
 Засор канал                              1
 Name: count, Length: 66, dtype: int64)

In [247]:
c_dataset_v2.Group.value_counts(), c_dataset_v2.Cat.value_counts()

(Group
 Управление домом     25762
 ГВС                   1179
 Домофоны              1072
 Лифты                  942
 Водоотведение            5
 Благоустройство          5
 Ливневки                 3
 Сантехника               2
 Бродячие животные        2
 Отопление                2
 Газоснабжение            1
 ХВС                      1
 Канализация              1
 Дороги                   1
 Name: count, dtype: int64,
 Cat
 Сантехника                                       9710
 Общие вопросы                                    3442
 Канализация                                      2926
 ГВС (Несоответствие температурных параметров)    2443
 Электроэнергия                                   2300
                                                  ... 
 Засор канал                                         1
 Общие вопросы ХВС                                   1
 Кровля(очистка от снега и наледи)                   1
 Подключение к диспетчеризации                       1
 Опломбирование пр